<a href="https://colab.research.google.com/github/riyasingh07/CNVeM/blob/master/CNVeM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CNVeM**


##Imports

In [ ]:
import math
import time
import sys
!pip install parse
from parse import parse

  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=9f1c3ae2dba7ea4f17a1050441f8c57ad2dcced16e9ed67d5044969db305cb95
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built parse


##Necessary data structures

In [ ]:
class Signal:
  def __init__(self, name="", maxsig=[], lrr=[],sigA=[],sigB=[], gt=[],kn=0,inf=[],sufreq=0.0,mdc=0,noh=0,hap=[],md=[],prior=0.0):
    self.name=name
    self.maxsig=maxsig 
    self.lrr=lrr
    self.sigA=sigA
    self.sigB=sigB
    self.gt=gt
    self.kn=kn
    self.inf=inf
    self.sufreq=sufreq
    self.mdc=mdc
    self.noh=noh
    self.hap=hap
    self.md=md
    self.prior=prior

In [ ]:
class cnv:
 def __init__(self,name=""	,typ=0	,pstart=0	,pend=0	,sstart=0	,snps=0	,estart=0	,esnps=0	,cprobs=[]	,carr=0.0	,blr=0.0	,po=0 ,initfr=0.0):
  self.name=name
  self.typ=typ
  self.pstart=pstart
  self.pend=pend
  self.sstart=sstart
  self.snps=snps
  self.estart=estart
  self.esnps=esnps
  self.cprobs=cprobs
  self.carr=carr
  self.blr=blr
  self.po=po
  self.initfr=initfr

##Some values

In [ ]:
R2p=2.506628  
Op=0  
Lrr=0  
Bknown=0 
Gte_flag=0 
AddAn_flag=0 
Prior=0.0  
Colsk=0  
V=0  
Outlieflag=0  
Qc=1.0  
Version=1.000  

##CNVeM code

In [ ]:
def cnvem(argc,argv):
  data=[]
  cnvs=[]
 
  names=None
  sdat= []
  indiv=[4001]
  sind=[0]
  snps=[12]
  subs=[0]
  steps=[30]
  infile="data.csv"
  posfile="cnvfile.txt"
  mapfile="mapfile.txt"
  sumfile="sumout.csv"
  fm=[1]
  lm=[0]
  nocnv=[1]
  reps=[100]
  diff=[]
  seed=time.time() 
  cnvfreq=[0.0435]
  noc=[None]
  lrr_flag=[0]
  thresh=[0.0]
  best_start=[None]
  best_end=[None]
  noca=[None]
  blr=[None]
  minl=[None]
  scr=[0]
  initfr=[0.0]
  Version=1.0
  print("CNVEM By Riya Kumari Singh\t\t Version "+str(Version))

  icl(argc,argv,indiv, snps,subs,steps,reps,sind,infile,cnvfreq,lrr_flag,thresh,fm,lm,posfile,mapfile,minl)
  initial_out(seed, indiv[0], snps[0], steps[0], lrr_flag[0], thresh[0], subs[0],reps[0])
  
  if(posfile!="nofile"):
    cnvs=readpos(posfile, nocnv,indiv[0],mapfile)
    
  for j in range(nocnv[0]):
    print("\nCNV "+str(j+1))
    snps[0]=lm[0]-fm[0]+1
    if(posfile!="nofile"):
      fm[0]=cnvs[j].sstart
      snps[0]=cnvs[j].snps
      lm[0]=fm[0]+snps[0]-1
	
    if(snps[0]>0):
      
      data=makedarr(indiv[0],snps[0])
      print("Memory allocated\n")

      readdata(indiv[0],data,snps[0],infile,fm[0])
      
      if(names==None and posfile!="nofile"):
        names=make2arr(indiv[0],30)
        for k in range(indiv[0]):
          names[k]=data[k].name
        
      snps[0]=remove_empty_markers(data,snps[0],indiv[0])

      print("Data read in "+str(indiv[0] )+" individuals "+str(snps[0] )+" markers "+str(fm[0] )+" -"+str(lm[0]  )+" \n")
      if(snps[0]>0):
        
        if(Bknown==0 or minl[0]>=snps[0]):
          
          blr[0]=bound_an(steps[0],data,indiv[0], snps[0],lrr_flag[0],best_start,best_end,snps[0],scr,initfr)
          best_start[0]=0
          best_end[0]=snps[0]
          blr[0]-=calc_bl(data,indiv[0],0,snps[0])
        
        else:
          blr[0]=bound_an(steps[0],data,indiv[0], snps[0],lrr_flag[0],best_start,best_end,minl, scr,initfr)

        print("EM finished\n")

        if(thresh[0]>0.0):
          threshold(indiv[0],data,thresh[0])
          
        noca[0]=results(indiv[0],data,snps[0])
  
        if(AddAn_flag>0):
          calc_stuff(data,snps[0],indiv[0])

        if(posfile!="nofile"):

          if(nocnv[0]>0):
            cnvs[j].estart=best_start[0]
            cnvs[j].esnps=best_end[0]-best_start[0]
            cnvs[j].carr=noca[0]
            cnvs[j].blr=blr[0]
            for k in range(indiv[0]):
              cnvs[j].cprobs[k]=data[k].inf[1]
            cnvs[j].po=scr[0]
            cnvs[j].initfr=initfr[0]
            
          else:
            cnvs[j].estart=0
            cnvs[j].esnps=0
            cnvs[j].carr=0.0
            cnvs[j].blr=0.0
          
        #Subsampling

        if(subs[0]>0):
          
          threshold(indiv[0],data,0.8)
          diff=[0.0 for i in range(reps[0])]
          sdat=makedarr(indiv[0],subs[0])
          print("subsampling\n")
          
          for i in range(reps[0]):
            
            subsample(data,sdat[0],indiv[0],snps[0],subs[0])
            bound_an(steps[0],data,indiv[0], snps[0],lrr_flag[0],best_start,best_end,snps[0], scr,initfr)
            
            if(thresh[0]>0.0):
              threshold(indiv[0],sdat[0],thresh[0])
            
            diff[i]=compare(data,sdat[0],indiv[0])
            print("Simulated dataset "+str(i )+":\t"+str(diff[i]  )+"\n")
        
          calcss(diff, reps[0])
          
        if (sind[0]>0) :
          
          threshold(indiv[0],data,0.8)
          diff=[0.0 for i in range(reps[0])]
          sdat=makedarr(sind[0],snps[0])
          print("subsampling\n")
          
          for i in range(reps[0]):
            
            subsample2(data,sdat[0],indiv[0],sind[0],snps[0],cnvfreq)
            bound_an(steps[0],data,indiv[0], snps[0],lrr_flag[0],best_start,best_end,snps[0], scr,initfr)
            
            if(thresh[0]>0.0):
              threshold(sind[0],sdat[0],thresh[0])
            
            diff[i]=compare2(data,sdat[0],indiv[0],sind[0])
            print("Simulated dataset "+str(i )+":\t"+str(diff[i] )+"\n")
          
          calcss(diff, reps[0])
          
  if(posfile!="nofile"):
    summout(sumfile,cnvs,indiv[0],nocnv[0],names)
  print("Program terminating\n")
  

##EM

In [ ]:
def runem(steps, data, indiv, snps, start, end, noc, lrr_flag, rm, prfreq, estf) :
  
  prior=[0.0,0.0]
  gte_prior=[0.0,0.0]
  
  mu = make2arr(snps, 4)
  var = make2arr(snps, 4)

  defv = initialize_em(indiv, snps, data, start, end, prior, gte_prior, prfreq, estf) #default variance

  rm[0]=0
  while True:
    for i in range (steps):
      if (lrr_flag == 0):
        estimate_distrib(indiv, mu, var, data, start, end, defv)

      else:
        estimate_distrib_lrr(indiv, snps, mu, var, data, start, end)

      if (Gte_flag == 0):
        noc = estimate_status(indiv, mu, var, data, start, end, lrr_flag, prior)

      else:
        noc = estimate_status_gte(indiv, mu, var, data, start, end, lrr_flag, prior, gte_prior)

     
      if (Op == 1) :
        ll = calculate_lik(indiv, snps, mu, var, data, start, end, lrr_flag)
        print("Step "+str(i)+"  loglikelihood "+str(ll )+" carriers "+str(noc )+"\n")
      
  

    sc = sanity_check(indiv, start, end, data, mu)

    if (sc == 1 and Outlieflag == 1):
      rm[0] = 0
      for i in range (indiv):
        if (data[i].inf[1] > 0.5) :
          data[i].kn = -1
          print(str(data[i].name)+" removed\n")
          rm[0]+=1

        
      else:
        if (data[i].kn == 0):
          rn = RandDbl()
          data[i].inf[0] = rn
          data[i].inf[1] = 1 - rn
      
      print(str(rm[0])+" outliers removed\n")
    
    if (sc == 1 and Outlieflag == 0):
      rm[0] = 1
      sc = 0
    if sc!=1:
      break
  ll = calculate_lik(indiv, snps, mu, var, data, start, end, lrr_flag)

  return ll


In [ ]:
def initialize_em(indiv, snps, data, start, end, prior, gte_prior, meanf, estf):
  c = 0
  sh = 0
  rcount = 0
  ret = 0.0
  ms = 0.0

  meanf[0] = 0.0
  
  for i in range(start,end):
    meanf[0] += estf[i]
    meanf[0] = meanf[0] / (end - start)
  count = 0
  for k in range(indiv):
    data[k].kn = 0
    mdc = 0
    
    for i in range(start,end):
      if (data[k].gt[i] == 1):
        data[k].kn+=1
      if (data[k].gt[i] == 3):
        mdc+=1
    
    if (mdc == end - start) :
      data[k].kn = -1
      rcount+=1
      
    for i in range(start,end):
      if (data[k].gt[i] == 0 or data[k].gt[i] == 2) :
        ms += data[k].maxsig[i]
        ret += pow(data[k].maxsig[i], 2)
        count+=1

  ret = ret / count - pow(ms / count, 2)

  count = 0
  for i in range(indiv):
    
    if (data[i].kn == 0):
      rn = RandDbl()
      data[i].inf[0] = rn
      data[i].inf[1] = 1 - rn
      c+=1
    else:
      data[i].inf[0] = 1.0
      data[i].inf[1] = 0.0
      
    if (data[i].kn == 1):
      sh+=1
    if (data[i].kn >= 0):
      count+=1
  
  c=.1
  print(str(c)+" individuals with unknown deletion status\n")
  print(str(rcount)+" individuals removed for missing data\n")
  if (Op == 1):
    print(str(c)+" individuals with unknown deletion status\n")
    if (Gte_flag == 1):
      print(str(sh)+" single heterozygotes\n")
  
  if (Prior > 0) :
    prior[1] = (Prior * count) / c
    if (prior[1] > 0.5):
      prior[1] = 0.5

  else:
    hemf = 2 * (meanf[0]) * (1 - meanf[0])
    prior[1] = (hemf * count) / c
    
    if (hemf < 0.01):
      prior[1] = (0.01 * count) / c
    if (prior[1] > 0.5) :
      prior[1] = 0.5

  prior[0] = (1 - prior[1])
  print("Modified priors: "+str(prior[0])+" "+str(prior[1]))
  pofc = prior[1] * c / pow(0.99, end - start)
  
  sh=.3
  gte_prior[1] = pow(0.99, end - start - 1) * 0.01 * (end - start) * pofc / sh
  gte_prior[0] = 1 - gte_prior[1]

  return ret

In [ ]:
def sanity_check(indiv, start, end,data, mu):
  
  c=[0.0,0.0]
  asd=[0.0,0.0]
  ret = 0
  
  for k in range(2):
    asd[k] = 0
    c[k] = 0
    for i in range(indiv):
      if (data[i].kn >= 0):
        for j in range(start,end):
          if (data[i].gt[j] == 0 or data[i].gt[j] == 2)  :
            asd[k] += data[i].maxsig[j] * data[i].inf[k]
            c[k] += data[i].inf[k]
           

    if (c[k] > 0):
      asd[k] = asd[k] / c[k]
   

  if (asd[1] > asd[0] and c[1] > 0.0001):
    print("Sanity check failed:\t")
    for k in range(2):
      print(( k, asd[k], c[k]))
    print("\n")
    ret = 1
   
  return ret

In [ ]:
def estimate_status( indiv, mu,var,data,start, end, lrr_flag, prior):
  lik=[0.0,0.0]
  ret = 0.0
  lprior=[0.0,0.0]

  for i in range(2):
    lprior[i] = math.log(prior[i])

  for i in range(indiv):
    if (data[i].kn == 0):
      lik[0] = lprior[0]
      lik[1] = lprior[1]

      for j in range(start,end):
        for k in range(2):
          if (data[i].gt[j] != 3):
            if (lrr_flag == 0):
              lik[k] += logprob(data[i].maxsig[j], mu[j][k + data[i].gt[j]],var[j][k + data[i].gt[j]])
            else:
              lik[k] += logprob(data[i].lrr[j], mu[0][k],var [0][k]) + lprior[k]
            if not (lik[k] > -1e50):
              print(("Problem:", j, k, data[i].maxsig[j], mu[j][k + data[i].gt[j]],var [j][k + data[i].gt[j]], logprob(data[i].maxsig[j], mu[j][k + data[i].gt[j]],var[j][k + data[i].gt[j]])))
          
      for k in range(2):
        lik[k] = math.exp(lik[k])

      if (lik[1] >= 0 and lik[0] >= 0):
        if (lik[0] + lik[1] > 0):
          lik[0] = lik[0] / (lik[1] + lik[0])
        else:
          print("Underflow error in estimate_status\n")
          lik[0] = 0.5
        

      else:
        print(("ERROR:", lik[0], lik[1]))
        for j in range(start,end):
          for k in range(2):
            print((j, k, mu[j][k + data[i].gt[j]],var[j][k + data[i].gt[j]]))
        sys.exit("----Failed---")
      
      data[i].inf[0] = lik[0]
      data[i].inf[1] = 1 - lik[0]
      ret += data[i].inf[1]
  
  return ret


In [ ]:
def estimate_status_gte(indiv, mu, var, data, start, end, lrr_flag, prior, gte_prior):
  lik=[0.0,0.0]
  ret = 0.0
  lprior=[0.0,0.0]
  l_gte_prior=[0.0,0.0]

  for i in range(2):
    lprior[i] = log(prior[i])
    l_gte_prior[i] = log(gte_prior[i])
  

  for i in range(indiv):
    if (data[i].kn < 2 and data[i].kn >= 0):
      if (data[i].kn == 0):
        for k in range(2):
          lik[k] = lprior[k]

      else:
        for k in range(2):
          lik[k] = l_gte_prior[k]
          
      for j in range(start,end):
        if (data[i].gt[j] != 3 and data[i].gt[j] != 1):
          for k in range(2):
            if (lrr_flag == 0):
              lik[k] += logprob(data[i].maxsig[j], mu[j][k + data[i].gt[j]], var [j][k + data[i].gt[j]])
            else:
              lik[k] += logprob(data[i].lrr[j], mu[0][k], var [0][k])

      for k in range(2):
        lik[k] = exp(lik[k])

      if (lik[1] >= 0 and lik[0] >= 0):
        if (lik[0] + lik[1] > 0):
          lik[0] = lik[0] / (lik[1] + lik[0])
        else:
          print("Underflow error in estimate_status_gte\n")
          lik[0] = 0.5
        

      else:
        print(("ERROR:", lik[0], lik[1]))
        for j in range(start,end):
          for k in range(2):
            print(( j, k, mu[j][k + data[i].gt[j]],var[j][k + data[i].gt[j]]))
        sys.exit("----Failed---")
      

      data[i].inf[0] = lik[0]
      data[i].inf[1] = 1 - lik[0]
      ret += data[i].inf[1]
    

  return ret


In [ ]:
def estimate_distrib(indiv, mu, var, data, start, end, defv):

  c = []
  for j in range(end):
    c.append([0.0 for i in range(4)])

  for j in range(start,end):

    for k in range(2):
      mu[j][k] = 0.0
      mu[j][k + 2] = 0.0
      var [j][k] = 0.0
      var [j][k + 2] = 0.0
      c[j][k] = 0.0
      c[j][2 + k] = 0.0

      for i in range(indiv):
        if ((data[i].gt[j] == 0 or data[i].gt[j] == 2) and data[i].kn >= 0):
          mu[j][k + data[i].gt[j]] += data[i].maxsig[j] * data[i].inf[k]
          c[j][data[i].gt[j] + k] += data[i].inf[k]
        
      if (c[j][0 + k] > 0):
        mu[j][k] = mu[j][k] / c[j][k]
      if (c[j][2 + k] > 0):
        mu[j][k + 2] = mu[j][k + 2] / c[j][2 + k]

      for i in range(indiv):
        if (data[i].gt[j] == 0 or data[i].gt[j] == 2 and data[i].kn >= 0):
          var [j][k + data[i].gt[j]] += pow(data[i].maxsig[j] - mu[j][k + data[i].gt[j]], 2) * data[i].inf[k]

      if (c[j][k] > 1):
        var [j][k] = math.sqrt(var [j][k] / (c[j][k] - 1))
      else:
        var [j][k] = 0.0
      if (c[j][2 + k] > 1):
        var [j][k + 2] = sqrt(var [j][k + 2] / (c[j][2 + k] - 1))
      else:
        var [j][k + 2] = 0.0
  
    fix_variances(c, var, j, defv)

  if (Op == 1):
    for k in range(2):
      print("State "+str(k)+" Mean: ")
      for j in range(start,end):
        print(( j, mu[j][k], mu[j][k + 2]))

      print("\n")
      print("State "+str(k)+" stdvs: ")
      for j in range(start,end):
        print((j, var[j][k], var[j][k + 2]))
      print("\n")
      print("State "+str(k)+" Counts: ")
      for j in range(start,end):
        print(( j, c[j][k], c[j][k + 2]))
      print("\n")

In [ ]:
def fix_variances(c, var, m, defv):
  
  relf=[[0.0,0.0],[0.0,0.0],[0.0,0.0]]
  relt = 10
  fac = 0.3

  for i in range(0,3,2):
    for j in range(2):
      if (c[m][i + j] >= relt):
        relf[i][j] = 1
      else:
        relf[i][j] = 0

  if (relf[0][0] + relf[2][0] == 0):
    
    for i in range(0,3,2):
      for j in range(2):
        var [m][j + i] = defv
  else:
    if (relf[2][0] == 0 and var [m][2] < fac *var [m][0]):
      var [m][2] = fac * var [m][0]

    if (relf[0][0] == 0 and var [m][0] < fac * var [m][2]):
      var [m][0] = fac * var [m][2]

    if (relf[0][1] == 0 and var [m][1] < fac * var [m][0]):
      if (relf[0][0] == 1):
        var [m][1] = fac * var [m][0]
      else:
        var [m][1] = fac * var [m][2]

    if (relf[2][1] == 0 and var [m][3] < fac * var [m][2]):
      if (relf[2][0] == 1):
        var [m][3] = fac * var [m][2]
      else:
        var [m][3] = fac * var [m][0]

In [ ]:
def estimate_distrib_lrr(indiv, snps, mu, var, data, start, end):
  for k in range(2):
    mu[0][k] = 0.0
    var [0][k] = 0.0
    c = 0.0

    for i in range(indiv):
      for j in range(start,end):
        if (data[i].gt[j] == 0 or data[i].gt[j] == 2 and data[i].kn >= 0):
          mu[0][k] += data[i].lrr[j] * data[i].inf[k]
          var [0][k] += pow(data[i].lrr[j], 2) * data[i].inf[k]
          c += data[i].inf[k]
        
    if (c > 0):
      mu[0][k] = mu[0][k] / c
      var [0][k] = math.sqrt(var [0][k] / c - pow(mu[0][k], 2))
    else:
      var [0][k] = 1e-10
  

In [ ]:
def calculate_lik(indiv, snps, mu, var, data, start, end, lrr_flag):
  
  ret = 0.0
  count=[0.0,0.0,0.0,0.0]
  
  for j in range(start,end):
    for i in range(indiv):
      if (data[i].kn >= 0):
        if (data[i].gt[j] != 1 and data[i].gt[j] != 3):
          sum = 0.0
          ts = 0.0
          for k in range(2):
            if (lrr_flag == 0):
              sum += data[i].inf[k] * prob(data[i].maxsig[j], mu[j][k + data[i].gt[j]],var [j][k + data[i].gt[j]])
            else:
              sum += data[i].inf[k] * prob(data[i].lrr[j], mu[0][k],var [0][k])
            count[k + data[i].gt[j]] += data[i].inf[k]
            ts += data[i].inf[k]
          
          if (sum == 0.0 or ts < 0.99 or ts > 1.01):
            print("Warning: Error in calculate_lik (sum="+str(sum)+", ts="+str(ts)+"), count= "+str(count)+"\n")
            sys.exit("----Failed---")
          else:
            ret += math.log(sum)
        
    if (V == 1):
      for k in range(4):
        print("marker "+str(j)+": genotype "+str(k)+" count "+str(count[k])+" , "+str(count[k]+1)+" mu "+str(mu[j][k])+" "+str(mu[j][k+1])+" sig "+str(var [j][k])+" "+str(var [j][k+1])+" \n")

  return ret

In [ ]:
def correct_lik(indiv, data, start, end, ll):
  
  ret = 0.0
  c1 = 0
  c2 = 0

  for j in range(start,end):
    for i in range(indiv):
      if (data[i].gt[j] != 1 and data[i].gt[j] != 3):
        if (data[i].kn >= 0):
          c1+=1
        else:
          c2+=1
  print("correction factor "+str(1.0 + c2 / c1))
  ret = ll / c1 * (c2 + c1)

  return ret

##Finding Boundaries

In [ ]:
class conf:
  def __init__(self, ll=0, start=0, end=0,cs=[0], noc=0, sc=0):
    self.ll=ll
    self.start=start
    self.end=end
    self.cs=cs
    self.noc=noc
    self.sc=sc 

def bound_an(steps, data, indiv, snps, lrr_flag, bs, be, minl, rsc, initfr) :
  
  max = 0
  noe = 0
  maxs = 400

  estf = [0.0 for i in range (snps)]
  calc_estf(indiv, 0, snps, data, estf)

  noe = (snps - minl + 1) * (snps - minl + 2) / 2

  print("Maximizing Boundaries, minimum length "+str(minl)+", number of elements "+str(noe)+"\n")

  j = 0
  s1 = 1
  s2 = 1
  while (noe > maxs):
    noe = noe / 2
    j+=1

  while (j > 0):
    if (j > 0):
      j-=1
      s1+=1
    
    if (j > 0):
      j-=1
      s2+=1
    
  if (s1 > 1):
    print("To reduce computation, only rough borders are calculated, reduced iterations to steps of "+str(s1)+str(s2))
  noe = 0
  for b in range (0,snps-minl+1,s1):
    for e in range (b + minl,snps+1,s2):
      noe+=1
  
  all = [conf() for i in range(noe + 1) ]
  for i in range(noe + 1):
    all[i].cs = [0.0 for k in range(indiv)]

  bl = calc_bl(data, indiv, 0, snps)
  print("Baseline "+str(bl))

  all[0].start = 0
  all[0].end = snps
  all[0].ll = runem(steps, data, indiv, snps, 0, snps, [all[0].noc], lrr_flag, [all[0].sc], initfr, estf)

  for j in range(indiv):
    all[0].cs[j] = data[j].inf[0]
  print("Full length "+str(all[0].ll))

  i = 1
  for b in range (0,snps-minl+1,s1):
    for e in range (b + minl,snps+1,s2):
      all[i].start = b
      all[i].end = e
      all[i].ll = runem(steps, data, indiv, snps, b, e, [all[i].noc], lrr_flag, [all[i].sc], initfr, estf)

      all[i].ll += calc_bl(data, indiv, 0, b)
      all[i].ll += calc_bl(data, indiv, e, snps)
      if (Outlieflag > 0):
        all[i].ll = correct_lik(indiv, data, 0, snps, all[i].ll)
      if (all[i].noc < 0.01 and all[i].ll > bl + 100):
        V = 1
        print("This is more than weird error\n")
        print("ll "+str(all[i].ll )+"> bl "+str(bl)+"\n")
        print("noc="+str(all[i].noc)+"\n"+str(b)+"-"+str(e)+"\n")
        
      for j in range(indiv):
        all[i].cs[j] = data[j].inf[0]
      
      if (all[i].ll > all[max].ll):
        max = i
      i+=1

  print("highest likelihood "+str(all[max].ll)+"("+str(all[max].sc)+") markers "+str(all[max].start)+"-"+str(all[max].end)+"\n")

  ret = all[max].ll - bl
  for i in range(noe):
    err = 0.0
    for j in range(indiv):
      err += abs(all[i].cs[j] - all[max].cs[j])
 
  bs[0] = all[max].start
  be[0] = all[max].end
  if (noe > 1):
    runem(steps, data, indiv, snps, all[max].start, all[max].end,[all[i].noc], lrr_flag, rsc, initfr, estf)
  
  return ret

In [ ]:
def calc_estf( indiv, start, end,data, estf):
  gt=[0,0,0]

  for i in range(start,end):
    gt[0]=0
    gt[1]=0
    gt[2]=0
    count=0
    for k in range(indiv):
      if(data[k].gt[i]<3):
        gt[data[k].gt[i]]+=1
        count+=1
        
    if(count>0 and gt[1]==0):
      gt[1]+=1
      count+=1
      
    if(count>0):
      estf[i]=gt[0]/count*gt[2]/gt[1]-gt[1]/(4.0*count)
      estf[i]=estf[i]/(1.0+estf[i])
      print("Estimated frequency Marker "+str(i)+"  "+str(estf[i])+"\n")

In [ ]:
def calc_bl(data, indiv, start, end):

  mu=[0.0,0.0,0.0,0.0]
  var=[0.0,0.0,0.0,0.0]
  c=[0,0,0,0]
  ll = 0.0
  mdtest = 0

  for j in range(start,end):
    for k in range(4):
      mu[k] = 0.0
      var [k] = 0.0
      c[k] = 0
    
    tot = 0
    for i in range(indiv):
      if (data[i].kn >= 0):
        tot+=1
        c[data[i].gt[j]]+=1
        if (data[i].gt[j] == 0 or data[i].gt[j] == 2):
          mu[data[i].gt[j]] += data[i].maxsig[j]
          var [data[i].gt[j]] += pow(data[i].maxsig[j], 2)
      

    for k in range(0,3,2):
      if (c[k] > 0):
        mu[k] = mu[k] / c[k]
        if (var [k] / c[k] - pow(mu[k], 2) >= 0):
          var [k] = math.sqrt(var [k] / c[k] - pow(mu[k], 2))
        else:
          print("variance error: "+str(var [k] / c[k])+" "+str(pow(mu[k], 2))+"\n")
          var [k] = 0.0
        
        if (var [k] == 0.0 and c[k] > 1):
          var[k]=0.37
          
          for i in range(indiv):
            if (data[i].kn >= 0):
              if (data[i].gt[j] == k):
                print(( i, data[i].maxsig[j]))
        
        if (V == 1 or Op == 1):
          print("Baseline calculations SNP "+str(j)+": gt "+str(k)+" count "+str(c[k])+" mu "+str(mu[k])+" sigma "+str(var[k])+" \n")
      
    if (c[0] == 1 or var [0] == 0.0):
      var [0] = var [2]
    if (c[2] == 1 or var [2] == 0.0):
      var [2] = var [0]
    if (c[2] > 1 or c[0] > 1):
      for i in range(indiv):
        if (data[i].gt[j] != 1 and data[i].gt[j] != 3):
          ll += logprob(data[i].maxsig[j], mu[data[i].gt[j]], var [data[i].gt[j]])
          if (ll <= -1e50):
            print("Error in calculating likelihood for boundary configuration\n")
            print("marker "+str(j + 1)+" individual "+str(i+1)+" "+str(ll)+"\n")
            print("Counts: Gt0 "+str( c[0])+" Gt1 "+str( c[1])+" Gt2 "+str( c[2])+" Gt3 "+str( c[3])+" ="+str(tot)+" \n")
            print("Genotype "+str(data[i].gt[j])+"\tsignal "+str(data[i].maxsig[j])+"\tmu "+str(mu[data[i].gt[j]])+"\tsigma "+str(var [data[i].gt[j]])+"\tlik "+str(prob(data[i].maxsig[j]))+"\n")
            sys.exit("----Failed---")
  return ll

##Memory handling

In [ ]:
def make2arr(d1,d2):
  ret = []
  for i in range(d1):
    ret.append([0 for i in range(d2)])
  return ret


In [ ]:
def makedarr( d1, d2):
  ret=[]
  for j in range(d1):
    t=Signal()
    t.maxsig = [0.0 for i in range(d2)]
    t.gt = [0 for i in range(d2)]
    t.sigA = [0.0 for i in range(d2)]
    t.sigB = [0.0 for i in range(d2)]
    t.lrr = [0.0 for i in range(d2)]
    t.name = ''
    t.inf = [0.0,0.0]
    t.kn = 0
    ret.append(t)
  
  return ret

In [ ]:
def makecnvarr( d1, d2):
  ret=[]
  for i in range(d1):
    t=cnv()
    t.cprobs =[0.0 for j in range(d2)]
    t.name = ''
    t.pstart = 0
    t.pend = 0
    t.po = 0
    ret.append(t)
  return ret

In [ ]:
def prob( signal, mu, sig):
  sig=.8
  return math.exp(-pow((signal-mu)/sig,2)/2)/(R2p*sig)

In [ ]:
def logprob(signal,mu, sig):
  return (-pow((signal-mu)/sig,2)/2)-math.log(R2p*sig)

In [ ]:
def RandDbl():
  import random
  return random.random()

##I/O

In [ ]:
def initial_out(seed, indiv, snps, steps, lrr_flag, thresh, subs, reps):
  print("Seed: \t\t\t"+str(seed)+"\n")
  print("number of individuals\t"+str(indiv)+"\n")
  print("number of markers\t"+str(snps)+"\n")
  print("number of EM-steps\t"+str(steps)+"\n")
  if (lrr_flag < 0):
    print("analyzing LRR as well\n")
  if (Gte_flag > 0):
    print("Modelling genotyping error\n")
  if (thresh > 0.0):
    print("Calling threshold "+str(thresh)+"\n")

  if (subs > 0):
    print("resampling:\n")
    print("number of resampled SNPs\t"+str(subs)+"\n")
    print("number of subsamples taken\t"+str(reps)+"\n")

In [ ]:
def icl(noa, argum, indiv, snps, subs, steps, reps, sind, infile,  cnvfreq, lrr_flag,  thresh, first, last, posfile, mapfile, minl):
  
  while ((noa > 1) and argum[1][0] == '-'):
    if  argum[1][1]=='F':
      first[0] = int(argum[1][2:])
      
    elif  argum[1][1]== 'L':
      last[0] = int( argum[1][2:])
      
    elif  argum[1][1]== 'b':
      posfile= argum[1][2:]
      
    elif  argum[1][1]== 'm':
      mapfile= argum[1][2:]
      
    elif  argum[1][1]== 'e':
      steps[0] = int(argum[1][2:])
      
    elif  argum[1][1]== 's':
      indiv[0] = int(argum[1][2:])
      
    elif  argum[1][1]== 'p':
      snps[0] = int(argum[1][2:])
      
    elif  argum[1][1]== 'u':
      subs[0] = int(argum[1][2:])
      
    elif  argum[1][1]== 'r':
      sind[0] = int(argum[1][2:])
      
    elif  argum[1][1]== 'n':
      #number of subsamples taken*/
      reps[0] = int(argum[1][2:])
      
    elif  argum[1][1]== 'S':
      Op = 1
      
    elif  argum[1][1]== 'i':
      #inputfile*/
      infile= argum[1][2:]
      
    elif  argum[1][1]== 'f':
      #frequency of  the CNV in resampled datasets
      cnvfreq[0] = float(argum[1][2:])
      
    elif  argum[1][1]== 'B': #determine best boundaries
      Bknown = 1
      minl[0] = int(argum[1][2:])
      
    elif  argum[1][1]== 'R': #use LRR for EM
      lrr_flag[0] = 1
      Lrr = -1
      
    elif  argum[1][1]== 'G': #model genotyping error
      Gte_flag = 1
      
    elif  argum[1][1]== 'P': #prior
      Prior = float( argum[1][2:])
      
    elif  argum[1][1]== 'T': #prior
      thresh[0] = float( argum[1][2:])
      
    elif  argum[1][1]== 'A': #calculate distributions of hybridization intensity
      AddAn_flag = 1
      
    elif  argum[1][1]== 'O': # only a single hybridization value is available
      Lrr = 1
      
    elif  argum[1][1]== 'c': #skip this many columns in the beginning of the input file
      Colsk = int(argum[1][2:])
      
    elif  argum[1][1]== 'D': #remove potential outliers an reanalyze
      Outlieflag = 1
      
    elif  argum[1][1]== 'Q': #set maximim number of missing data per SNP
      Qc = float(argum[1][2:])
      
    elif  argum[1][1]== 'h': #Help
      help_output()
      
    else:
      print("Unknown option "+argum[1]+"\n")
      help_output()
      sys.exit("----Failed---")
    
    noa+=1
    argum=argum[1:]
  

  if ( subs[0] > snps[0]):
    subs[0] > snps[0]
  if ( last[0] < 1 or last[0] > snps[0]):
    last[0] = snps[0]

In [ ]:
def help_output():
  print("\nAnalysis options:\n")
  print("  ex:\tRounds of EM \n")
  print("  Bx:\tdetermine best boundaries x is minimum length	\n")
  print("  R:\tuse LogR Ratio for EM\n")
  print("  G:\tmodel genotyping error\n")
  print("  Px:\tprior \n")
  print("  Tx:\tthreshold x individuals with prob >x are called as carriers\n")
  print("  F:\tFist SNP to be considered\n")
  print("  L:\tLast SNP to be considered\n")
  print("  Qx:\tset maximim percentage of missing data per SNP\n")
  print("\nThe Dataset:\n")
  print("  ix:\tinputfile \n")  
  print("  sx:\tSample Size \n")
  print("  px:\tNumber of SNPs\n")
  print("  b:\tInputfile for multiple CNV-borders\n")
  print("  m:\tMapfile for SNPs\n")
  print("  O:\tOnly one hybridization intensity is in the datafile\n")
  print("  cx:\tNumber of columns with irrelevant info\n")
  print("\nOutput\n")
  print("  S:\tsilent EM off\n") 
  print("  A:\tCalculate distributions of hybridization intensity\n")
  print("  h:\tHelp-this output\n")
  print("\nSubsampling\n")
  print("  ux:\tnumber of subsampled snps  \n")
  print("  rx:\tsubsampled sample size \n")
  print("  nx:\tnumber of subsamples taken \n")
  print("  fx:\tfrequency of  the CNV in resampled datasets \n")
  print("\n")	
  sys.exit("----Failed---")

In [ ]:
def readmap( mapfile,nom) :
  inf = open(mapfile, "r")
  print("map positions taken from "+mapfile+"\n")
  nom[0] = 0

  i = 0
  temp = inf.readline()
  while temp:
    if temp[0]== '#':
      pass #comments, ignore
    elif temp[0]== 'P':
      nom[0] = int(temp[1:])
      ret = [0.0 for k in range(nom[0])]
      print(str(nom[0])+" Markers in map file\n")
      
    else:
      if (nom[0] == 0) :
        print("Error in "+mapfile+" P-line missing\n")
        sys.exit("----Failed---")
      ret[i] = int(temp)
      i+=1
    temp = inf.readline()
  
  if (i != nom[0]) :
    print("Error: Wrong number of markers in "+mapfile+" ("+str(i)+"!="+str(nom[0])+")\n")
    sys.exit("----Failed---")
  
  #testing if map is ordered
  for i in range(nom[0]-1):
    if (ret[i + 1] < ret[i]) :
      print("Error: map file "+mapfile+" not ordered (Line "+str(i+2)+" ("+str(ret[i + 1])+"<"+str(i+2)+" "+str(ret[i])+"))\n")
      sys.exit("----Failed---")
    
  return ret

In [ ]:
def readpos(posfile, nocnv, indiv, mapfile) :
  inf = open(posfile, "r")
  i = 0
  nom=[0]
  if (mapfile!= "nofile"):
    map = readmap(mapfile,nom)

  print("CNV boundaries taken from "+posfile+"\n")
  nocnv[0] = 0
  temp = inf.readline()
  while temp:
    if temp[0]== '#':
      pass #comments, ignore
    elif temp[0]== 'P':
      nocnv[0] = int( temp[1])
      ret = makecnvarr( nocnv[0], indiv)
      print(""+str(nocnv[0])+" CNVs in input file\n")
      
    elif temp[0]== 'S':
      if ( nocnv[0] == 0) :
        print("Error in "+posfile+" P-line missing\n")
        sys.exit("----Failed---")
      x=temp[1:].split(',')
      ret[i].typ=int(x[0])
      ret[i].sstart=int(x[1])
      last=int(x[2])
      ret[i].snps = last - ret[i].sstart + 1
      i+=1
      
    elif temp[0]=='L':
      if not nocnv[0]:
        print("Error in "+posfile+" P-line missing\n")
        sys.exit("----Failed---")
      
      if not map:
        print("Error: No map file provided\n")
        sys.exit("----Failed---")
      x=temp[1:].split(',')
      
      ret[i].typ=int(x[0])
      ret[i].pstart=int(x[1])
      ret[i].pend=int(x[2])

      j = 0
      while (map[j] < ret[i].pstart and j < nom[0]) :
        j+=1
      
      if (j == nom) :
        print("WARNING CNV "+str(i+1)+" not covered by SNPs\n")
        ret[i].sstart = 0
        ret[i].snps = 0
      else :
        ret[i].sstart = j + 1
        while ( j < nom[0] and map[j] <= ret[i].pend) :
          j+=1
        
        if (j + 1 == ret[i].sstart) :
          print("WARNING: CNV "+str(i + 1)+" not covered by SNPs\n")
          ret[i].snps = 0
        else:
          ret[i].snps = j - ret[i].sstart + 1
        print(" "+str(ret[i].sstart)+"-"+str(ret[i].snps)+"\n")
        i+=1
    temp = inf.readline()
  
  if (i != nocnv[0]) :
    print("Wrong number of regions in "+posfile+"\n")
    sys.exit("----Failed---")
  return ret

In [ ]:
def threshold(indiv, data, thresh):
  for i in range(indiv):
    if(data[i].inf[1]>thresh):
      data[i].inf[1]=1.0
      data[i].inf[0]=0.0
    
    else:
      data[i].inf[0]=1.0
      data[i].inf[1]=0.0

In [ ]:
def results( indiv, data, nos):
  c = 0.0
  for i in range(indiv):
    if (data[i].kn < 0):
      print(data[i].name+"\t******\t******")
    else:
      print((data[i].name, data[i].inf[0], data[i].inf[1]))
    if (data[i].kn > 0 and data[i].inf[1] > 0.01):
      print("\t")
      for j in range(nos):
        print(data[i].gt[j])
    print("\n")
    if (data[i].kn < 0):
      data[i].inf[1] = -1
    else:
      c += data[i].inf[1]
      
  print("Expected number of deletions "+str(c)+"\n")


In [ ]:
def readdata(indiv, data, nsnps, infile, first) :
  inf = open(infile, "r")
  ts = indiv * (4 * 40) + 50
  noc = 3 - Lrr
  gt=[0,0,0,0]
  name=""
  print("Opening "+infile+"\n")
  print(indiv)
  temp = ''
  #Read first line
  while True:
    temp=inf.readline()
    if (temp[0] != '#'):
      break

  sl = len(temp)
  #first line is IDs
  a = 0
  k = 0

  k = jump_comma(k, temp, sl, Colsk) #first 5 items are uninteresting
  while (k < sl) :
    j = k
    name=''
    while (temp[k] != ',' and temp[k] != '.') :
      name+= temp[k]
      k+=1
    data[a].name=name
    k = jump_comma(k, temp, sl, noc)
    a+=1
  
  if (a != indiv) :
    print("Input error, wrong number of individuals (observed: "+str(a)+" expected "+str(indiv)+") in first line \n")
    print("There may also be a format error in the input file in the first line\n")
    sys.exit("----Failed---")
  
  # other lines contain hybridization
  for i in range(first): #skip all snps until the first
    while True:
      temp=inf.readline()
      if (temp[0] != '#'):
        break
      
  c = 0
  while (c < nsnps) :
    
    sl = len(temp)
    a = 0
    k = 0

    k = jump_comma(k, temp, sl, Colsk)
    for i in range(4):
      gt[i] = 0

    while (k < sl) :
      
      if Lrr==0:
        k = read_2hsignal(temp, k, data, a, c, sl)
        
      elif Lrr==-1:
        k = read_Lrrsignal(temp, k, data, a, c, sl)
        
      elif Lrr== 1:
        k = read_1hsignal(temp, k, data, a, c, sl)

      else:
        print("We should never be here!!!\n")
        sys.exit("----Failed---")
      gt[data[a].gt[c]]+=1
      a+=1
    

    '''if (a != indiv) :
      print("Input error, wrong number of individuals ("+str(a)+") in line "+str(c+2)+"\n")
      print("There may also be a format error in the input file at marker ("+str(a)+") in line "+str(c+2)+"\n")
      sys.exit("----Failed---")'''
    
    print("Marker "+str()+": ", c)
    for i in range(4):
      print(gt[i])
    if (Qc * indiv < gt[3]) :
      print("Marker "+str(c)+" failed QC\n")
      for i in range(indiv):
        data[i].gt[c] = 3
    
    c+=1
    temp=inf.readline()

In [ ]:
def read_1hsignal(temp, k, data, a, c, sl) :
  j=parse("{}{},{}", temp[k:])
  if j!=None :
    g1=j[0]
    g2=j[1]
    i1=float(j[2].split(',')[0])
    data[a].maxsig[c] = i1

    if (g1 == g2):
      if (g1 == 'A'):
        data[a].gt[c] = 0
      else:
        data[a].gt[c] = 2
    else:
      if (g1 == 'N') :
        data[a].gt[c] = 3
        data[a].maxsig[c] = 0.0
      else :
        data[a].gt[c] = 1
        data[a].maxsig[c] = 0.0
    k = jump_comma(k, temp, sl, 2)
  else:
    k = sl + 3
  return k

In [ ]:
def read_2hsignal(temp, k, data, a, c, sl) :
  j=parse("{}{},{},{}", temp[k:])
  if j!=None:
    g1=j[0]
    g2=j[1]
    i1=float(j[2])
    i2=float(j[3].split(',')[0])
    if (g1 == g2):
      if (g1 == 'A'):
        data[a].gt[c] = 0
        data[a].maxsig[c] = i1
      else :
        data[a].gt[c] = 2
        data[a].maxsig[c] = i2
    else:
      if (g1 == 'N') :
        data[a].gt[c] = 3
        data[a].maxsig[c] = 0.0
      else :
        data[a].gt[c] = 1
        data[a].maxsig[c] = 0.0
      
    data[a].sigA[c] = i1
    data[a].sigB[c] = i2

    k = jump_comma(k, temp, sl, 3)

  else:
    k = sl + 3
  return k

In [ ]:
def read_Lrrsignal(temp, k, data, a, c, sl) :
  
  j=parse("{}{},{},{},{}", temp[k:])
  if j!=None:
    g1=j[0]
    g2=j[1]
    i1=float(j[2])
    i2=float(j[3])
    i3=float(j[4].split(',')[0])
    if (g1 == g2):
      if (g1 == 'A') :
        data[a].gt[c] = 0
        data[a].maxsig[c] = i1
      else :
        data[a].gt[c] = 2
        data[a].maxsig[c] = i2
    else:
      if (g1 == 'N') :
        data[a].gt[c] = 3
        data[a].maxsig[c] = 0.0
      else :
        data[a].gt[c] = 1
        data[a].maxsig[c] = 0.0
    data[a].sigA[c] = i1
    data[a].sigB[c] = i2
    data[a].lrr[c] = i3

    k = jump_comma(k, temp, sl, 4)
  else:
    k = sl + 3
  return k

In [ ]:
def jump_comma( k, temp, sl, nos):
  j=0
  while j<nos:
    if (k >= sl):
      break
    if (temp[k] == ','):
      j+=1
    k+=1
  return k

In [ ]:
def summout(sumfile, cnvs, indiv, nocnv, names) :
  sout = open(sumfile, "w")
  
  sout.write("No,Type,Start,End,FirstSNP,length,estimatedFS,estimatedlength")
  
  sout.write( "\n")
  for j in range(nocnv) :
    if j not in [4,5]:
      sout.write( str(j+1)+","+str(cnvs[j].typ)+","+str(cnvs[j].pstart)+","+str(cnvs[j].pend))
      sout.write( ","+str(cnvs[j].sstart)+","+str(cnvs[j].snps)+","+str(cnvs[j].estart)+","+str(cnvs[j].esnps))
    
      sout.write( "\n")
  sout.close()


##Additional analyses

In [ ]:
def calc_stuff( res, snps, indiv):
  print("\n------------------------------------------\n")
  print("Analysis of signal distribution\n")
  for i in range(snps):
    for j in range(4):
      print("GT "+str(j))
      calc_mv(res, i, j, indiv)
    print("\n------------------------------------------\n")

In [ ]:
def calc_mv(data, snp, gt, indiv):
  
  for k in range(2):
    muA = 0.0
    varA = 0.0
    muB = 0.0
    varB = 0.0
    muR = 0.0
    varR = 0.0
    c = 0.0

    for i in range(indiv):
      if (data[i].gt[snp] == gt):
        muA += data[i].sigA[snp] * data[i].inf[k]
        muB += data[i].sigB[snp] * data[i].inf[k]
        if (Lrr == -1):
          muR += data[i].lrr[snp] * data[i].inf[k]
          varR += pow(data[i].lrr[snp] * data[i].inf[k], 2)
        
        c += data[i].inf[k]
      
    muA = muA / c
    muB = muB / c
    muR = muR / c
    varR = math.sqrt(varR / c - pow(muR, 2))

    for i in range(indiv):
      if (data[i].gt[snp] == gt):
        varA += pow(data[i].sigA[snp] - muA, 2) * data[i].inf[k]
        varB += pow(data[i].sigB[snp] - muB, 2) * data[i].inf[k]
      
    varA = math.sqrt(varA / c)
    varB = math.sqrt(varB / c)

    print((muA, varA, muB, varB))
    if (Lrr == -1):
      print(( muR, varR))
    print(c)

In [ ]:
def subsample(data, sdat, indiv, snps, subs):
  samp = [0 for i in range(subs)]

  for k in range(indiv):
    sdat[k].name= data[k].name
    sdat[k].kn = 0
    sdat[k].inf[0] = data[k].inf[0]
    sdat[k].inf[1] = data[k].inf[1]
  
  for i in range(subs):
    while True:
      j = snps * RandDbl()
      for k in range(i):
        if (j == samp[k]):
          j = -1
      if(j != -1):
        break
    samp[i] = j
    print( j)
    
    for k in range(indiv):
      sdat[k].maxsig[i] = data[k].maxsig[j]
      sdat[k].sigA[i] = data[k].sigA[j]
      sdat[k].sigB[i] = data[k].sigB[j]
      sdat[k].gt[i] = data[k].gt[j]
  for k in range(indiv):
    for i in range(subs):
      if (sdat[k].gt[i] == 1):
        sdat[k].kn+=1

In [ ]:
def subsample2(data, sdat, indiv,sind,snps,cnvfreq):
  
  hmz = cnvfreq * sind + 0.5

  #Edit to subsample a fixed percentage of hemizygous.

  print(""+str(hmz)+" hemizygous ("+str(cnvfreq)+", "+str(sind)+")\n")
  for k in range(sind - hmz):
    while True:
      j = indiv * RandDbl()
      if (data[j].inf[0] >= 0.5):
        break

    sdat[k].name= data[j].name
    sdat[k].kn = data[j].kn
    sdat[k].inf[0] = data[j].inf[0]
    sdat[k].inf[1] = data[j].inf[1]

    for i in range(snps):
      sdat[k].maxsig[i] = data[j].maxsig[i]
      sdat[k].sigA[i] = data[j].sigA[i]
      sdat[k].sigB[i] = data[j].sigB[i]
      sdat[k].gt[i] = data[j].gt[i]

  for  k in range(sind):
    while True:
      j = indiv * RandDbl()
      if (data[j].inf[0] <= 0.5):
        break

    sdat[k].name=data[j].name
    sdat[k].kn = data[j].kn
    sdat[k].inf[0] = data[j].inf[0]
    sdat[k].inf[1] = data[j].inf[1]

    for i in range(snps):
      sdat[k].maxsig[i] = data[j].maxsig[i]
      sdat[k].sigA[i] = data[j].sigA[i]
      sdat[k].sigB[i] = data[j].sigB[i]
      sdat[k].gt[i] = data[j].gt[i]

In [ ]:
def compare(data,sdat,indiv):
  stat=0.0
  for i in range(indiv):
    stat+=abs(data[i].inf[0]-sdat[i].inf[0])
  return stat

In [ ]:
def compare2(data, sdat, indiv, sind):
  stat = 0.0
  tot = 0

  for i in range(sind):
    j = 0
    while (data[j].name!= sdat[i].name):
      j+=1
    stat += abs(data[j].inf[0] - sdat[i].inf[0])
    tot += data[j].inf[0]
  
  print((stat, tot))
  if (tot > sind):
    sys.exit("----Failed---")
    pass
  return stat

In [ ]:
def calcss(results, noe):
  av = 0.0
  tvar = 0.0
  min = results[0]
  max = results[0]
  pcount = 0

  for i in range(noe):
    av += results[i]
    tvar += results[i] * results[i]
    if (results[i] > max):
      max = results[i]
    if (results[i] < min):
      min = results[i]
    if (results[i] < 0.0001):
      pcount+=1
  av = av / noe
  tvar = sqrt(tvar / noe - pow(av, 2))

  print("average "+str(av)+" standard deviation "+str(tvar)+" minimum "+str(min)+" maximum "+str(max)+" perfect sets "+str(pcount)+"\n")

In [ ]:
def remove_empty_markers( data, snps, indiv):
  ret = snps
  i=0
  while i <ret:
    count = 0
    for j in range(indiv):
      if (data[j].gt[i] == 3):
         count+=1
    if (count == indiv) :
      print("Removing Marker "+str(i))
      for k in range(snps - 1):
        for j in range(indiv):
          data[j].maxsig[k] = data[j].maxsig[k + 1]
          data[j].lrr[k] = data[j].lrr[k + 1]
          data[j].sigA[k] = data[j].sigA[k + 1]
          data[j].sigB[k] = data[j].sigB[k + 1]
          data[j].gt[k] = data[j].gt[k + 1]
          
        ret-=1
      i-=1
    i+=1
      
    
  return ret

## RUNNER

In [ ]:
def runCNVeM():
  argv=input("Insert the command as per manual:\n").split()
  argv=['']+argv
  argc=len(argv)
  cnvem(argc,argv)

In [ ]:
runCNVeM()
#-e20 -B4 -idata.csv -s4 –F137655983 –L137875956 -p10

Insert the command as per manual:
-e20 -B4 -idata.csv -s4 –F137655983 –L137875956 -p10
CNVEM By Riya Kumari Singh		 Version 1.0
Seed: 			1646519991.5965264

number of individuals	4

number of markers	12

number of EM-steps	20

map positions taken from mapfile.txt

10 Markers in map file

CNV boundaries taken from cnvfile.txt

8 CNVs in input file

 9-2


 1-0


 9-0


CNV 1
Memory allocated

Opening data.csv

4
Marker :  0
3
1
0
0
Marker :  1
3
1
0
0
Marker :  2
2
2
0
0
Data read in 4 individuals 3 markers 1 -3 

Estimated frequency Marker 0  -0.06666666666666667

Estimated frequency Marker 1  -0.06666666666666667

Estimated frequency Marker 2  -0.14285714285714285

Maximizing Boundaries, minimum length 3, number of elements 1.0

(1, 0.569)
(3, 0.569)
Baseline 12.983458047990426
0.1 individuals with unknown deletion status

0 individuals removed for missing data

Modified priors: 0.6000000000000001 0.39999999999999997
Full length -5.569768958410565
0.1 individuals with unknown deletion